In [2]:
!pip install tmdbv3api


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api.exceptions import TMDbException
import random
import pandas as pd
import csv
tmdb=TMDb()
tmdb.api_key='9cf68f4c97c8f0cc6bb9646da389a808'
tmdb.language='en'
movie=Movie()

In [15]:
full_data = pd.read_csv('merged_data_mod_scores_filtered.csv', header=0, index_col=0)
full_data

,Running Time,Genres,IMDB Title,MPAA,Domestic Distributor,Domestic Opening,Earliest Release Date,IMDB Budget,TMDB Budget,Genre ID,...,Release Year,Merged Budget,Merged Revenue,Raw Star Scores,Unweighted Star Score,Simple Weight Star Score,Log Weight Star Score,Exponential Weight Star Score,director_scores,production_company_scores
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2,1 hr 49 min,Action Crime Drama Mystery Thriller,Broken City (2013),R,Twentieth Century Fox,8268908.0,"January 18, 2013 (Domestic, EMEA)",35000000,56000000,"[53, 80, 18]",...,2013,35000000,19701164,"[295, 158, 21, 0, 0, 0, 0, 0, 80, 0, 0, 0, 0, ...",554,470.0,459.478850,444.130435,67.0,360.0
6,1 hr 29 min,Action Adventure Comedy Horror Sci-Fi Thriller,Spiders (2013),PG-13,Millennium Entertainment,NaN,"February 8, 2013 (Domestic)",0,7000000,"[878, 53]",...,2013,7000000,257019,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0.0,0.000000,0.000000,0.0,262.0
8,1 hr 56 min,Action Crime Drama Thriller,Out of the Furnace (2013),R,Relativity Media,5220288.0,"December 4, 2013 (Domestic)",22000000,22000000,"[53, 18, 80]",...,2013,22000000,11330849,"[165, 39, 0, 0, 0, 0, 59, 0, 0, 0, 0, 0, 0, 0,...",263,223.7,200.985775,204.695122,0.0,1896.0
9,1 hr 32 min,Documentary Music,One Direction: This Is Us (2013),PG,TriStar Pictures,15815497.0,"August 28, 2013 (EMEA)",10000000,10000000,"[99, 10402]",...,2013,10000000,28873374,"[0, 0, 0, 0, 0, 0, 0, 135, 0, 0, 0, 0, 0, 0, 0]",135,40.5,0.333804,12.500000,0.0,366.0
14,1 hr 40 min,Fantasy Horror Thriller,Mama (2013),PG-13,Universal Pictures,28402310.0,"January 17, 2013 (APAC, EMEA)",15000000,15000000,[27],...,2013,15000000,71628180,"[91, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",91,91.0,90.000188,91.000000,0.0,1514.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9734,1 hr 22 min,Comedy Horror,Vampires Suck (2010),PG-13,Twentieth Century Fox,12202831.0,"August 18, 2010 (Domestic, EMEA)",20000000,20000000,"[27, 35]",...,2010,20000000,36661504,[],0,0.0,0.000000,0.000000,NaN,NaN
9745,1 hr 59 min,Action Drama,Act of Vengeance (2010),NaN,NaN,NaN,"November 4, 2010 (Germany)",0,12000000,[18],...,2010,12000000,23423390,[],0,0.0,0.000000,0.000000,NaN,NaN
9750,1 hr 26 min,Comedy Drama,The Virginity Hit (2010),R,Sony Pictures Entertainment (SPE),301885.0,"September 24, 2010 (Domestic)",2000000,2000000,[35],...,2010,2000000,636706,[],0,0.0,0.000000,0.000000,NaN,NaN


Want to first get the list of TMDB IDs
then API call TMDB to get the US release dates if they exist
reformat those release dates in the same format we want
add that as column

In [5]:
int_id=98357
movie_info=movie.details(int_id)
if movie_info:
    release_dates=movie_info.get('release_dates',{})
    results=release_dates.get('results',[])
    print(results)
    for result in results:
            # THIS (BELOW) IS THE LINE ADDED FOR US RELEASE DATE ONLY
        if result.get('iso_3166_1')=='US':
            for release_date in result.get('release_dates',[]):
                if release_date.get('type')==3:
                    print(release_date.get('release_date'))

[{'iso_3166_1': 'US', 'release_dates': [{'certification': 'R', 'descriptors': {}, 'iso_639_1': '', 'note': '', 'release_date': '2013-01-18T00:00:00.000Z', 'type': 3}]}, {'iso_3166_1': 'IE', 'release_dates': [{'certification': '', 'descriptors': {}, 'iso_639_1': '', 'note': '', 'release_date': '2013-03-01T00:00:00.000Z', 'type': 3}]}, {'iso_3166_1': 'PT', 'release_dates': [{'certification': 'M/12', 'descriptors': {}, 'iso_639_1': '', 'note': '', 'release_date': '2013-02-21T00:00:00.000Z', 'type': 3}]}, {'iso_3166_1': 'AR', 'release_dates': [{'certification': '', 'descriptors': {}, 'iso_639_1': '', 'note': '', 'release_date': '2013-02-28T00:00:00.000Z', 'type': 3}]}, {'iso_3166_1': 'GR', 'release_dates': [{'certification': 'Κ-15', 'descriptors': {}, 'iso_639_1': '', 'note': '', 'release_date': '2013-03-28T00:00:00.000Z', 'type': 3}, {'certification': '', 'descriptors': {}, 'iso_639_1': '', 'note': 'DVD,Blu-ray', 'release_date': '2013-06-21T00:00:00.000Z', 'type': 5}]}, {'iso_3166_1': 'BE

In [6]:
tmdb_ids = full_data['TMDB ID']

us_release_dates_float = {}
for id in tmdb_ids:
    try:
        int_id = int(id)
        movie_info=movie.details(int_id)
        if movie_info:
            release_dates=movie_info.get('release_dates',{})
            results=release_dates.get('results',[])

            for result in results:
            # THIS (BELOW) IS THE LINE ADDED FOR US RELEASE DATE ONLY
                if result.get('iso_3166_1')=='US':
                    for release_date in result.get('release_dates',[]):
                        if release_date.get('type')==3:
                    #Write the movie ID to the list
                            us_release_dates_float[id] = release_date.get('release_date')
    except:
        pass
    

In [10]:
list(us_release_dates_float.keys())

[98357.0,
 164331.0,
 164457.0,
 164558.0,
 132232.0,
 132344.0,
 132363.0,
 198277.0,
 165739.0,
 165864.0,
 100542.0,
 100544.0,
 68179.0,
 133805.0,
 199373.0,
 232175.0,
 199534.0,
 101299.0,
 68721.0,
 68724.0,
 68727.0,
 68728.0,
 167032.0,
 68726.0,
 134374.0,
 134411.0,
 200155.0,
 4258.0,
 201132.0,
 70074.0,
 168672.0,
 201749.0,
 136400.0,
 169209.0,
 103731.0,
 202214.0,
 136795.0,
 202575.0,
 169813.0,
 137093.0,
 169881.0,
 72190.0,
 72359.0,
 137968.0,
 72525.0,
 72559.0,
 72710.0,
 203802.0,
 204082.0,
 138697.0,
 138832.0,
 138843.0,
 139038.0,
 106646.0,
 106747.0,
 205220.0,
 238302.0,
 172792.0,
 205601.0,
 107811.0,
 107846.0,
 239019.0,
 107985.0,
 140823.0,
 75612.0,
 75656.0,
 305242.0,
 174311.0,
 76170.0,
 76203.0,
 76285.0,
 109091.0,
 76338.0,
 174645.0,
 174671.0,
 174675.0,
 174772.0,
 76617.0,
 76640.0,
 109410.0,
 109414.0,
 109417.0,
 109418.0,
 109421.0,
 109424.0,
 109428.0,
 109431.0,
 109439.0,
 109443.0,
 109445.0,
 109451.0,
 109491.0,
 207871.0,


In [17]:
#us_release_df = pd.DataFrame.from_dict(us_release_dates_float)
us_release_df = pd.DataFrame({'TMDB ID': list(us_release_dates_float.keys()), 'US Release Date': list(us_release_dates_float.values())})
full_data_us = full_data.merge(us_release_df, 'left', left_on='TMDB ID', right_on='TMDB ID')
full_data_us

,Running Time,Genres,IMDB Title,MPAA,Domestic Distributor,Domestic Opening,Earliest Release Date,IMDB Budget,TMDB Budget,Genre ID,...,Merged Budget,Merged Revenue,Raw Star Scores,Unweighted Star Score,Simple Weight Star Score,Log Weight Star Score,Exponential Weight Star Score,director_scores,production_company_scores,US Release Date
0,1 hr 49 min,Action Crime Drama Mystery Thriller,Broken City (2013),R,Twentieth Century Fox,8268908.0,"January 18, 2013 (Domestic, EMEA)",35000000,56000000,"[53, 80, 18]",...,35000000,19701164,"[295, 158, 21, 0, 0, 0, 0, 0, 80, 0, 0, 0, 0, ...",554,470.0,459.478850,444.130435,67.0,360.0,2013-01-18T00:00:00.000Z
1,1 hr 29 min,Action Adventure Comedy Horror Sci-Fi Thriller,Spiders (2013),PG-13,Millennium Entertainment,NaN,"February 8, 2013 (Domestic)",0,7000000,"[878, 53]",...,7000000,257019,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,0.0,0.000000,0.000000,0.0,262.0,2013-02-08T00:00:00.000Z
2,1 hr 56 min,Action Crime Drama Thriller,Out of the Furnace (2013),R,Relativity Media,5220288.0,"December 4, 2013 (Domestic)",22000000,22000000,"[53, 18, 80]",...,22000000,11330849,"[165, 39, 0, 0, 0, 0, 59, 0, 0, 0, 0, 0, 0, 0,...",263,223.7,200.985775,204.695122,0.0,1896.0,2013-11-09T00:00:00.000Z
3,1 hr 32 min,Documentary Music,One Direction: This Is Us (2013),PG,TriStar Pictures,15815497.0,"August 28, 2013 (EMEA)",10000000,10000000,"[99, 10402]",...,10000000,28873374,"[0, 0, 0, 0, 0, 0, 0, 135, 0, 0, 0, 0, 0, 0, 0]",135,40.5,0.333804,12.500000,0.0,366.0,2013-08-30T00:00:00.000Z
4,1 hr 40 min,Fantasy Horror Thriller,Mama (2013),PG-13,Universal Pictures,28402310.0,"January 17, 2013 (APAC, EMEA)",15000000,15000000,[27],...,15000000,71628180,"[91, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",91,91.0,90.000188,91.000000,0.0,1514.0,2013-01-18T00:00:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3372,1 hr 22 min,Comedy Horror,Vampires Suck (2010),PG-13,Twentieth Century Fox,12202831.0,"August 18, 2010 (Domestic, EMEA)",20000000,20000000,"[27, 35]",...,20000000,36661504,[],0,0.0,0.000000,0.000000,NaN,NaN,2010-08-18T00:00:00.000Z
3373,1 hr 59 min,Action Drama,Act of Vengeance (2010),NaN,NaN,NaN,"November 4, 2010 (Germany)",0,12000000,[18],...,12000000,23423390,[],0,0.0,0.000000,0.000000,NaN,NaN,2010-11-17T00:00:00.000Z
3374,1 hr 26 min,Comedy Drama,The Virginity Hit (2010),R,Sony Pictures Entertainment (SPE),301885.0,"September 24, 2010 (Domestic)",2000000,2000000,[35],...,2000000,636706,[],0,0.0,0.000000,0.000000,NaN,NaN,2010-09-24T00:00:00.000Z
3375,1 hr 20 min,Drama Fantasy Horror,Evidence of a Haunting (2010),NaN,Nocturnal Features,16800.0,"August 26, 2011 (Domestic)",0,25000,[27],...,100000,29904,[],0,0.0,0.000000,0.000000,NaN,NaN,NaN


In [18]:
full_data_us.to_csv('merged_data_mod_scores_filtered_US.csv')